In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("anper.csv")

In [4]:
df.loc[df.subcategoria == "Azulejos", "categoria"] = "Bizcocho"

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1072 entries, 0 to 1071
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   categoria     1072 non-null   object
 1   subcategoria  1072 non-null   object
 2   nombre        1072 non-null   object
 3   precio        1072 non-null   object
 4   url           1072 non-null   object
 5   descripcion   737 non-null    object
 6   tienda        1072 non-null   object
dtypes: object(7)
memory usage: 58.8+ KB


In [24]:
df.categoria.unique()

array(['Arcillas', 'tercer-fuego', 'Complementos', 'Horno',
       'Cristalizaciones', 'Elementos protección', 'Engobes', 'Esmaltes',
       'Herramientas', 'Hornos', 'Libros y cursos', 'Maquinaria',
       'Materiales para hornear', 'Materias primas',
       'Moldes y accesorios', 'Óxidos Colorantes', 'Pinceles',
       'Pirometría', 'Rotulador Cerámico', 'Tornos', '3er Fuego'],
      dtype=object)

In [63]:
df.loc[df.categoria == "3er Fuego", "subcategoria"] = "tercer-fuego"
df.loc[df.categoria == "Complementos", "categoria"] = 'Varios'


In [67]:
df = df.drop(columns=['descripcion'])

In [72]:
df['categoria'] = df['categoria'].astype('category')
df['tienda'] = df['tienda'].astype('category')

In [97]:
df.to_csv("anper.csv", index=False)

In [78]:
def procesar_precios(precio):
    if not precio:  # Verifica si la cadena está vacía
        return [('N/A', 0.0)]
    
    # Dividir los valores por '\n'
    items = precio.split('\n')
    result = []
    
    if len(items) > 1:
        for i in range(0, len(items), 2):
            if i+1 < len(items):
                cantidad = items[i]
                try:
                    precio_valor = float(items[i+1].replace('€', '').replace(',', '.'))
                except ValueError:
                    precio_valor = 0.0  # Maneja valores no numéricos o mal formateados
                result.append((cantidad, precio_valor))
    else:
        # Si no tiene el formato esperado, asumir que es un precio único
        try:
            precio_valor = float(precio.replace('€', '').replace(',', '.'))
        except ValueError:
            precio_valor = 0.0  # Maneja valores no numéricos o mal formateados
        result.append(('N/A', precio_valor))  # 'N/A' para indicar que no hay cantidad específica
    
    return result

# Aplicar la función a la columna precio
df['precios_procesados'] = df['precio'].apply(procesar_precios)

# Crear un nuevo DataFrame con las cantidades y precios
filas = []
for idx, row in df.iterrows():
    for cantidad, precio in row['precios_procesados']:
        filas.append({'Cantidad': cantidad, 'Precio': precio})

In [80]:
df_exploded = df.explode('precios_procesados')

In [82]:
df_exploded[['Cantidad', 'precio_final']] = pd.DataFrame(df_exploded['precios_procesados'].tolist(), index=df_exploded.index)

In [87]:
df_exploded['nombre'] = df_exploded.apply(
    lambda row: f"{row['nombre']} {row['Cantidad']}" if row['Cantidad'] != 'N/A' else row['nombre'], axis=1
)

In [96]:
df = df_exploded

In [85]:
df.precio = df.precio_final

In [94]:
df_exploded = df_exploded.drop(columns=['precio_final', 'Cantidad', 'precios_procesados'])

In [ ]:
df_exploded[df_exploded.categoria == "Arcillas"]

In [36]:
df.categoria.unique()

array(['Arcillas', 'Bizcocho', 'Horno', 'Varios', 'Materias primas',
       'Engobes', 'Esmaltes', 'tercer fuego', 'Herramientas',
       'Libros y cursos', 'Maquinaria', 'Óxidos Colorantes',
       'Pinceles/Espátulas pintor', 'Lápices/Tizas cerámicas'],
      dtype=object)

In [73]:
df_categorias = df[df['categoria'] == 'Maquinaria']

In [74]:
df_categorias.sample(10)

,categoria,subcategoria,nombre,precio,url,tienda
1078,Maquinaria,Cortadora refractario,MQ007093 CORTADORA REFRACTARIO DR-350 ZERO DUST,1139.86,https://anper.net/productos/941/maquinaria/mq0...,Anper
1098,Maquinaria,Pistolas,EP007616 Pistola Premium 474 1.0,103.21,https://anper.net/productos/872/maquinaria/ep0...,Anper
1586,Maquinaria,Tornos y laminadoras,MQ007075 Plato torno Forns G-1 D.300mm,59.29,https://anper.net/productos/615/tornos/mq00707...,Anper
1090,Maquinaria,Galleteras - Extrusionadoras,"MQ007130 Boquilla para Galletera ""C"" NRA-04 - ...",88.45,https://anper.net/productos/1132/maquinaria/MQ...,Anper
1099,Maquinaria,Pistolas,EP007617 Pistola 472 y Compressor 777,246.00,https://anper.net/productos/871/maquinaria/ep0...,Anper
1085,Maquinaria,Galleteras - Extrusionadoras,MQ007082 Extrusionadora pistola manual - con 6...,44.77,https://anper.net/productos/618/maquinaria/mq0...,Anper
1074,Maquinaria,Balanzas,"UT840301 Balanza digital lectura: 500g x 0,1g",82.28,https://anper.net/productos/8/maquinaria/ut840...,Anper
1093,Maquinaria,Laminadoras,Laminadora Nidec - Shimpo TSR sobremesa,780.45,https://anper.net/productos/505/maquinaria/lam...,Anper
1103,Maquinaria,Pulverizador Paintec,UT841025 Kit de 3 boquillas Paintec D.0.8 - 1....,14.40,https://anper.net/productos/866/maquinaria/ut8...,Anper
1080,Maquinaria,Galleteras - Extrusionadoras,Galletera Nidec - Shimpo NRA-04 Aluminio,5674.90,https://anper.net/productos/510/maquinaria/gal...,Anper


In [75]:
df_categorias.subcategoria.unique()

array(['Balanzas', 'Cortadora refractario',
       'Galleteras - Extrusionadoras', 'Laminadoras', 'Pistolas',
       'Pulverizador Paintec', 'Tornos y laminadoras'], dtype=object)

In [68]:
df.loc[df.subcategoria == "Engobe 1250ºC", "subcategoria"] = 'Engobes alta temperatura'
df.loc[df.subcategoria == "Engobe 980ºC", "subcategoria"] = 'Engobes baja temperatura'
df.loc[df.subcategoria == "Prodesco - EASP coloreados en suspensión", "subcategoria"] = 'Engobes en suspensión'
df.loc[df.subcategoria == "Engobe Vitrificable 1250ºC", "subcategoria"] = 'Engobes vitrificable  alta temperatura'
df.loc[df.subcategoria == "Engobe Vitrificable 980ºC", "subcategoria"] = 'Engobes vitrificable  baja temperatura'

In [53]:
df.loc[df.subcategoria == "Lápices", "categoria"] = 'Lápices/Tizas cerámicas'
df.loc[df.subcategoria == "Tizas", "categoria"] = 'Lápices/Tizas cerámicas'

In [ ]:
df.loc[1586, 'categoria'] = 'Torno'

In [35]:
df.loc[[1364, 1365], 'categoria'] = 'Materias primas'
df.loc[[1364, 1365], 'subcategoria'] = 'Vehículos y aglutinantes'